# Simplest 1D Gan
Simplest example based on this tutorial:
* [Blog post](http://blog.aylien.com/introduction-generative-adversarial-networks-code-tensorflow/)
* [Code](https://github.com/AYLIEN/gan-intro)
* [Reference of the reference](http://blog.evjang.com/2016/06/generative-adversarial-nets-in.html)

### Other References
* [Fantastic GANS and Where to find them](http://guimperarnau.com/blog/2017/03/Fantastic-GANs-and-where-to-find-them)
* [Karpathy Demo](http://cs.stanford.edu/people/karpathy/gan/)
* [Probabiliy Theory Basics](https://medium.com/@dubovikov.kirill/probabiliy-theory-basics-4ef523ae0820)
* [Ian Goodfellow Paper](https://arxiv.org/pdf/1406.2661.pdf)
* [How do GANs intuitively work](https://hackernoon.com/how-do-gans-intuitively-work-2dda07f247a1)
* [Mode Collapse](http://aiden.nibali.org/blog/2017-01-18-mode-collapse-gans/)
* [Gan Objective](http://aiden.nibali.org/blog/2016-12-21-gan-objective/)
* [Tensorflow sharing variables](https://www.tensorflow.org/programmers_guide/variable_scope)
* [BEGAN blog](https://blog.heuritech.com/2017/04/11/began-state-of-the-art-generation-of-faces-with-generative-adversarial-networks/)
* [BEGAN paper](https://arxiv.org/pdf/1703.10717.pdf)
* [BEGAN Tensorflow](https://github.com/carpedm20/BEGAN-tensorflow)
* [BEGAN Reddit](https://www.reddit.com/r/MachineLearning/comments/633jal/r170310717_began_boundary_equilibrium_generative/)
* [Veegan blog](https://akashgit.github.io/VEEGAN/)
* [Veegan paper](https://arxiv.org/pdf/1705.07761.pdf)
* [Unrolled Gans](https://arxiv.org/pdf/1611.02163.pdf)
* [F-Gan](https://arxiv.org/pdf/1606.00709.pdf)
* [Gans in Keras](https://github.com/eriklindernoren/Keras-GAN)

In [1]:
import numpy as np
import tensorflow as tf

# Include modules from other directories
import sys
sys.path.append('../tensorflow/')
import model_util as util
import models
import anim_util as anim

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

# Fix seed to reproduce same results
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

# Some meta parameters
start_lr = 0.005
decay = 0.95
num_steps = 5000
batch_size = 8
num_decay_steps = 150
logs_path = './logs'
save_dir = './save'
gpu_fraction = 0.1

# Delete logs directory if exist
if os.path.exists(logs_path):        
    os.system("rm -rf " + logs_path)

## Get MNIST Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Define DCGAN Model

In [3]:
model_gan = models.DCGAN(img_size=28, latent_size=100, training_mode=True)
D_real = model_gan.output_discriminator_real
D_fake = model_gan.output_discriminator_fake
model_trainable_vars = model_gan.trainable_variables
G = model_gan.output_generator
X = model_gan.discriminator_input_real
Z = model_gan.generator_input

### Define Loss function

In [4]:
with tf.variable_scope('loss_disc'):
    # Define losses
    loss_d = tf.reduce_mean(-tf.log(D_real) - tf.log(1 - D_fake))

with tf.variable_scope('loss_gen'):
    loss_g = tf.reduce_mean(-tf.log(D_fake))

### Get parameters from Generator and Discriminator

In [5]:
vars = model_trainable_vars
d_params = [v for v in vars if v.name.startswith('GAN/D/')]
g_params = [v for v in vars if v.name.startswith('GAN/G/')]

### Create the Session
Basically ask tensorflow to build the graph

In [6]:
# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

### Define the solver
We want to use the Adam solver to minimize or loss function.

In [7]:
def optimizer(loss, var_list, name='Solver'):
    # Solver configuration
    # Get ops to update moving_mean and moving_variance from batch_norm
    # Reference: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.name_scope(name):
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = start_lr
        # decay every 10000 steps with a base of 0.96
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                   num_decay_steps, decay, staircase=True)

        # Basically update the batch_norm moving averages before the training step
        # http://ruishu.io/2016/12/27/batchnorm/
        with tf.control_dependencies(update_ops):
            train_step = tf.train.AdamOptimizer(
                learning_rate).minimize(loss, global_step=global_step, var_list=var_list)
    
    return train_step, learning_rate


In [8]:
opt_disc, lr_disc = optimizer(loss_d, d_params, 'Solver_Disc')
opt_gen, lr_gen = optimizer(loss_g, g_params, 'Solver_Gen')

### Add some variables to tensorboard

In [9]:
# Create histogram for labels
tf.summary.image("in_disc_real", X, 4)
tf.summary.image("generator", G, 4)

# Monitor loss, learning_rate, global_step, etc...
tf.summary.scalar("loss_disc", loss_d)
tf.summary.scalar("loss_gen", loss_g)
tf.summary.scalar("lr_disc", lr_disc)
tf.summary.scalar("lr_gen", lr_gen)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Configure where to save the logs for tensorboard
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

### Initialize the values (Random values of weights)

In [10]:
# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

In [11]:
for step in range(num_steps):
    # Gather some real data and some latent z values    
    x_np = mnist.train.next_batch(batch_size)
    z_np = np.random.normal(0, 0.1, [batch_size, 100])
    
    # update discriminator
    sess.run([loss_d, opt_disc], {X: x_np,Z: z_np})

    # update generator
    z_np = gen.sample(batch_size)    
    sess.run([loss_g, opt_gen], {Z: z_np})
    
    # write logs at every iteration
    summary = merged_summary_op.eval(
        feed_dict={X: x_np,z: z_np})
    summary_writer.add_summary(summary, step)
        

ValueError: could not broadcast input array from shape (8,784) into shape (8)

In [ ]:
x_np = mnist.train.next_batch(batch_size)